# Bloomberg BQuant Spotlight Webinar Series: Technical Analysis

This is a companion notebook to the [Bloomberg BQuant Technical Analysis](https://blinks.bloomberg.com/screens/PLYR%20VOD%20331688234) webinar.

Before we get started, let's review some of the key concepts used in the examples below:

- The Slow Stochastic Oscillator is a momentum indicator that shows the location of the close relative to the high-low range over a set number of periods. The indicator can range from 0 to 100. Generally, the area above 80 indicates an overbought region, while the area below 20 is considered an oversold region.
- The Relative Strength Index (RSI), developed by J. Welles Wilder, is a momentum oscillator that measures the speed and change of price movements. The RSI oscillates between zero and 100. Traditionally the RSI is considered overbought when above 70 and oversold when below 30.

In [ ]:
# Import the required libraries
import bql
import bqviz as bqv
from bqplot import DateScale, OrdinalScale, ColorScale, Axis, GridHeatMap, Figure

import pandas as pd
from ipywidgets import VBox,  HBox, Label, SelectMultiple

from IPython.display import display, clear_output

import warnings
warnings.simplefilter('ignore')

In [ ]:
# Instantiate the BQL Service
bq = bql.Service()

# Define the BQL data request
request = """

    let(
        #date_range = range(-100d, -0d);

        #stochastics_series = TASS(dates=#date_range, fill='prev');
        
        #stochastics = last(#stochastics_series, 1).SSK;
        
        #rsi_series = rsi(dates=#date_range, fill='prev');
        
        #rsi = last(#rsi_series, 1);

        #price_series = dropna(px_last(dates=#date_range, fill='prev'));
                
    )

    get(
        #stochastics,
        #rsi,
        #rsi_series,
        #price_series,
        #stochastics_series,
        GICS_SECTOR_NAME

    )

    for(
        filter(
            members(['SXXP INDEX']),
            (
                market_cap > 15B
            )
        )   
    )
    
"""

# Execute the request
response = bq.execute(request)

# Generate a data frame from the response
df = pd.DataFrame()

df['RSI'] = response[1].df()['#rsi']
df['SSK'] = response[0].df()['#stochastics']

df.head(10)

In [ ]:
df.count()

In [ ]:
sector_categories = response[5].df()
sector_categories.head()

In [ ]:
sectors = sector_categories.GICS_SECTOR_NAME.unique()
sectors

In [ ]:
sector_selector = SelectMultiple(options=sectors.tolist(),value=sectors.tolist())
sector_selector

In [ ]:
scatter = bqv.InteractiveScatterPlot(df, hide_controls=True)

scatter.x_control.value = 'RSI'
scatter.y_control.value = 'SSK'

In [ ]:
pd.DataFrame(df['RSI']).head()

In [ ]:
RSI_hist = bqv.HistPlot(pd.DataFrame(df['RSI']))
SSK_hist = bqv.HistPlot(pd.DataFrame(df['SSK']))
RSI_hist.show()

In [ ]:
SSK_hist.show()

In [ ]:
left = response[2].df().reset_index()[['#rsi_series', 'DATE', 'ID']]
right = response[3].df().reset_index()[['#price_series', 'DATE', 'ID']]
right_2 =  response[4].df().reset_index()[['SSK', 'DATE', 'ID']]

merge_columns = ['ID','DATE']

series_df = pd.merge(left, 
                     right, 
                     how='left',  
                     left_on=merge_columns,
                     right_on=merge_columns
)

series_df = pd.merge(series_df, 
                     right_2, 
                     how='left',  
                     left_on=merge_columns,
                     right_on=merge_columns
).dropna()


series_df.set_index(['ID','DATE'],inplace=True)

series_df.head()

In [ ]:
rsi = series_df.reset_index().pivot('DATE','ID', '#rsi_series')
price = series_df.reset_index().pivot('DATE','ID', '#price_series')
SSK = series_df.reset_index().pivot('DATE','ID', 'SSK')
rsi.head()

In [ ]:
price_plot = bqv.InteractiveLinePlot(price,hide_controls=True, title="Price", legend=False)
price_plot.show()

In [ ]:
price_plot.line_control.value = [price.columns.values[0]]

In [ ]:
overbought_rsi_signal = series_df.reset_index().pivot('DATE','ID', '#rsi_series') >= 75
oversold_rsi_signal = series_df.reset_index().pivot('DATE','ID', '#rsi_series') <= 25

rsi_signal = overbought_rsi_signal * 1 + oversold_rsi_signal * -1

overbought_SSK_signal = series_df.reset_index().pivot('DATE','ID', 'SSK') >= 80
oversold_SSK_signal = series_df.reset_index().pivot('DATE','ID', 'SSK') <= 20

SSK_signal = overbought_SSK_signal * 1 + oversold_SSK_signal * -1


grid_data = pd.DataFrame([SSK_signal['AAK SS Equity'],rsi_signal['AAK SS Equity']],index=['SSK','RSI'])

grid_data

In [ ]:
x_sc, y_sc, col_sc = DateScale(domain=grid_data.T.index.values), OrdinalScale(), ColorScale(reverse=True, mid=0)

scales = {'column': x_sc, 'row': y_sc, 'color': col_sc}

y_lb = OrdinalScale(domain=['SSK','RSI'])

ax_x, ax_y = Axis(scale=x_sc), Axis(scale=y_lb, orientation='vertical')
grid_map = GridHeatMap(color=grid_data, scales=scales, null_color='#ffffff', stroke=None)

heatmap = Figure(marks=[grid_map] , axes=[price_plot.xaxis, ax_y], layout=price_plot.figure.layout)

heatmap

## Putting It All Together

We'll combine the components that we built above into a single UI that will allow us filter by sector and security.

In [ ]:
security_selector = price_plot.line_control
scatter_controls = VBox([ Label('Select a Sector:'), sector_selector], layout={'overflow_x': 'hidden'})
scatter_plot = VBox([scatter.show()])
hist_plot = VBox([RSI_hist.show(), SSK_hist.show()])
top_plots = HBox([scatter_plot, hist_plot])
top = HBox([scatter_controls, top_plots])

top_plots.layout.width = '100%'
scatter_controls.layout.width = '350px'
scatter_plot.layout.flex = '3 2'
hist_plot.layout.flex = '1 1'
scatter.figure.layout.width = '100%'
scatter.figure.layout.height = '600px'
scatter_margin = scatter.figure.fig_margin
scatter.figure.fig_margin['right'] = -10

In [ ]:
security_selector = price_plot.line_control

security_graph_box = VBox([price_plot.show(), heatmap], layout={'overflow_x': 'hidden'}) 
security_control_box = VBox([Label('Select Securities:'), security_selector])
bottom = HBox([security_control_box, security_graph_box])

security_selector.description = ''
security_graph_box.layout.flex = '4'
heatmap.layout.width = '100%'
price_plot.figure.layout.width = '100%'
scatter.figure.fig_margin = {'bottom': 40, 'left': 80, 'right': 0, 'top': 20}
price_plot.figure.fig_margin = {'bottom': 0, 'left': 55, 'right': 30, 'top': 20}
heatmap.fig_margin = {'bottom': 35, 'left': 55, 'right': 30, 'top': 0}

In [ ]:
def update(event):
    try:
        scatter._extract_points(event['new'])
        security_selector.options = scatter.selected_data.index.values
        security_selector.value = tuple(scatter.selected_data.index.values[0:min(4,len(event['new']))])
        RSI_hist.push(pd.DataFrame(scatter.selected_data['RSI']))
        SSK_hist.push(pd.DataFrame(scatter.selected_data['SSK']))
    except: 
        pass
    return True

def update_scatter(event):
    try:
        clear_output()
        selection = sector_categories.isin(event['new']).values
        new_data = df[selection]
        scatter.push(new_data)
        scatter.x_control.value = 'SSK'
        scatter.mark.observe(update,'selected')
        price_plot.line_control.observe(update_heatmap, 'value')
        display(top,bottom)
    except:
        pass
    return True
    
def update_heatmap(security):
    try:
        slicer = security['new']
        SSK = SSK_signal[[*slicer]].T.sum().T
        RSI = rsi_signal[[*slicer]].T.sum().T

        grid_data = pd.DataFrame([SSK, RSI], index=['SSK','RSI'])
        grid_map.color = grid_data
    except:
        pass
    return True

display(top,bottom)

scatter.mark.observe(update,'selected')
price_plot.line_control.observe(update_heatmap, 'value')
sector_selector.observe(update_scatter,'value')
    